In [ ]:
import torch
from features import ExtractWordEmbeddings
from nltk.tokenize import TweetTokenizer
from models.lstm import LSTMClassifier
tokenize = TweetTokenizer().tokenize
from preprocess_data import padBatch

Select a dimension based on these 10 dimensions

In [ ]:
dims = [
        'social_support',
        'conflict',
        'trust',
        'fun',
        'similarity',
        'identity',
        'respect',
        'romance',
        'knowledge',
        'power'
        ]

dim = 'INSERT DIMENSION HERE'
dim = 'conflict'
is_cuda = False # set to true only when using a GPU

weight_file = 'weights/LSTM/%s/best-weights.pth'%dim

In [ ]:
# load embeddings
em = ExtractWordEmbeddings(emb_type='glove')

# load model
model = LSTMClassifier(embedding_dim=300, hidden_dim=300)
state_dict = torch.load(weight_file)
model.load_state_dict(state_dict)
if is_cuda:
    model.cuda()

In [ ]:
sents = [
    'i trust you',
    'i believe in you',
    'believe me, that is not going to work',
    'i love you so much',
    'i hate you',
    "I don't love you any more",
    'i am proud of you',
    'i agree with that guy',
    'Thank you so much',
    'good to hear from you',
    'this is exactly what i wanted',
    'this is not what i wanted',
    'get off, you are wrong i do not'
]
tokenized_sents = [tokenize(sent) for sent in sents]
inputs = torch.tensor(padBatch([em.obtain_vectors_from_sentence(sent,True) for sent in tokenized_sents])).float()
if is_cuda:
    inputs = inputs.cuda()
outputs = torch.sigmoid(model(inputs)).tolist()

In [ ]:
for sent,score in zip(sents,outputs):
    print(round(score,2),sent)